### Experiment: Pre-processing

**Question**: Is it possible to train a model on unpreprocessed EEG data and still attain similar performance levels?

**Hypothesis**: The model will perform worse, but if still similar then the added value of not having to (manually) preprocess EEG data is very valuable and opens up a multitude of applications.

**Result**:

#### Part 1: Preparing data
To use hmp.utils.read_mne_data() and epoch the information, the files should be in .fif format, this replicates automated preprocessing as done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb excepting resampling to 100Hz

In [1]:
import mne
from pathlib import Path
import hsmm_mvpy as hmp
import pandas as pd
import numpy as np
import xarray as xr
from hmpai.data import add_stage_dimension

In [2]:
# Set up paths and file locations
data_path = Path("/mnt/d/thesis/sat1/")
behavioral_data_path = data_path / "ExperimentData/ExperimentData"
output_path = Path("../data/sat1/unpreprocessed")

subj_ids = [
    subj_id.name.split("-")[1][:4] for subj_id in (data_path / "eeg4").glob("*.vhdr")
]
subj_files = [
    str(output_path / f"unprocessed_{subj_id}_epo.fif") for subj_id in subj_ids
]
behavioral_files = [
    str(behavioral_data_path / f"{subj_id}-cnv-sat3_ET.csv") for subj_id in subj_ids
]

In [4]:
# Replacing preprocessing done in https://github.com/GWeindel/hsmm_mvpy/blob/main/tutorials/sample_data/eeg/0022.ipynb
# with only the necessary (non-manual) parts, like adding metadata for processing in HMP package, more info in link above
for subject_id in subj_ids:
    print(f"Processing subject: {subject_id}")
    subject_id_short = subject_id.replace("0", "")
    raw = mne.io.read_raw_brainvision(
        data_path / "eeg4" / f"MD3-{subject_id}.vhdr", preload=False
    )
    raw.set_channel_types(
        {"EOGh": "eog", "EOGv": "eog", "A1": "misc", "A2": "misc"}
    )  # Declare type to avoid confusion with EEG channels
    raw.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    raw.set_montage("standard_1020")  # Standard 10-20 electrode montage
    raw.rename_channels({"Fp1": "FP1", "Fp2": "FP2"})

    behavioral_path = behavioral_data_path / f"{subject_id}-cnv-sat3_ET.csv"
    behavior = pd.read_csv(behavioral_path, sep=";")[
        [
            "stim",
            "resp",
            "RT",
            "cue",
            "movement",
        ]
    ]
    behavior["movement"] = behavior.apply(
        lambda row: "stim_left"
        if row["movement"] == -1
        else ("stim_right" if row["movement"] == 1 else np.nan),
        axis=1,
    )
    behavior["resp"] = behavior.apply(
        lambda row: "resp_left"
        if row["resp"] == 1
        else ("resp_right" if row["resp"] == 2 else np.nan),
        axis=1,
    )
    # Merging together the exeperimental conditions info to have the format condition/stimulus/response
    behavior["trigger"] = (
        behavior["cue"] + "/" + behavior["movement"] + "/" + behavior["resp"]
    )
    # Filtering out < 300 and > 3000 Reaction times
    behavior["RT"] = behavior.apply(
        lambda row: 0
        if row["RT"] < 300
        else (0 if row["RT"] > 3000 else float(row["RT"]) / 1000),
        axis=1,
    )
    epochs = mne.io.read_epochs_fieldtrip(
        data_path / "eeg1" / f"data{subject_id_short}.mat", info=raw.info
    )
    epochs.rename_channels({"FP1": "Fp1", "FP2": "Fp2"})  # Naming convention
    epochs.set_montage("easycap-M1")
    epochs.filter(1, 35)  # Bandwidth filter from van Maanen, Portoles & Borst (2021)
    epochs.crop(tmin=-0.250)
    epochs.set_eeg_reference("average")
    epochs.metadata = behavior
    epochs.save(
        output_path / f"unprocessed_{subject_id}_epo.fif", overwrite=True, verbose=False
    )  # Saving EEG mne format

Processing subject: 0001
Extracting parameters from /mnt/d/thesis/sat1/eeg4/MD3-0001.vhdr...
Setting channel info structure...


/tmp/ipykernel_11057/4173686901.py:9: RuntimeWarning: The unit for channel(s) A1, A2 has changed from V to NA.
  raw.set_channel_types(
/tmp/ipykernel_11057/4173686901.py:13: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage("standard_1020")  # Standard 10-20 electrode montage


Not setting metadata
200 matching events found
No baseline correction applied
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    1.0s
[Parallel(n_job

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


...

Adding metadata with 6 columns
Processing subject: 0025
Extracting parameters from /mnt/d/thesis/sat1/eeg4/MD3-0025.vhdr...
Setting channel info structure...


/tmp/ipykernel_11057/4173686901.py:9: RuntimeWarning: The unit for channel(s) A1, A2 has changed from V to NA.
  raw.set_channel_types(
/tmp/ipykernel_11057/4173686901.py:13: RuntimeWarning: Not setting positions of 2 misc channels found in montage:
['A1', 'A2']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage("standard_1020")  # Standard 10-20 electrode montage


Not setting metadata
200 matching events found
No baseline correction applied
Setting up band-pass filter from 1 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 3041 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 3527 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done 4049 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 4607 tasks      | elapsed:    1.0s
[Parallel(n_job

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Adding metadata with 6 columns


In [5]:
output_path_data = Path("../data/sat1/data_unprocessed.nc")
# Run if data_unprocessed.nc does not exist or should be rewritten
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    verbose=False,
    subj_idx=subj_ids,
    rt_col="RT",
)
data.to_netcdf(output_path_data)

Processing participant ../data/sat1/unpreprocessed/unprocessed_0001_epo.fif's epoched eeg
198 trials were retained for participant ../data/sat1/unpreprocessed/unprocessed_0001_epo.fif
Processing participant ../data/sat1/unpreprocessed/unprocessed_0002_epo.fif's epoched eeg
200 trials were retained for participant ../data/sat1/unpreprocessed/unprocessed_0002_epo.fif
Processing participant ../data/sat1/unpreprocessed/unprocessed_0003_epo.fif's epoched eeg
191 trials were retained for participant ../data/sat1/unpreprocessed/unprocessed_0003_epo.fif
Processing participant ../data/sat1/unpreprocessed/unprocessed_0004_epo.fif's epoched eeg
200 trials were retained for participant ../data/sat1/unpreprocessed/unprocessed_0004_epo.fif
Processing participant ../data/sat1/unpreprocessed/unprocessed_0005_epo.fif's epoched eeg
190 trials were retained for participant ../data/sat1/unpreprocessed/unprocessed_0005_epo.fif
Processing participant ../data/sat1/unpreprocessed/unprocessed_0006_epo.fif's ep

#### Use information from stage_data to split unprocessed data

In [2]:
data_path = Path("../data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("../data/sat1/data_unprocessed.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

Finding stage changes
Combining segments


In [3]:
output_path = Path("../data/sat1/split_stage_data_unprocessed.nc")
output_data.to_netcdf(output_path)

### Part 2: Experiment

In [7]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import xarray as xr
from hmpai.utilities import print_results
from hmpai.pytorch.models import *
from hmpai.pytorch.training import k_fold_cross_validate
from hmpai.normalization import *
from hmpai.data import SAT1_STAGES_ACCURACY

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
logs_path = Path("../logs/exp_preprocessing/")
n_folds = 25
normalization_fn = norm_min1_to_1

##### 2a: Processed (control)

In [9]:
data_path = Path("../data/sat1/split_stage_data.nc")
data = xr.load_dataset(data_path)

In [10]:
model = SAT1GRU
model_kwargs = {
    "n_channels": len(data.channels),
    "n_samples": len(data.samples),
    "n_classes": len(data.labels),
}
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "processed"},
    "additional_name": f"preprocessing-processed",
    "labels": SAT1_STAGES_ACCURACY,
}
results = k_fold_cross_validate(
    model, model_kwargs, data, n_folds, train_kwargs=train_kwargs, normalization_fn=normalization_fn
)
print_results(results)

Fold 1: test fold: ['0009']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

...

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

Fold 25: Accuracy: 0.9221411192214112
Fold 25: F1-Score: 0.9206619349023638
Accuracies
[0.9412442396313364, 0.9265944645006017, 0.9324644549763034, 0.9143610013175231, 0.9544334975369458, 0.9079754601226994, 0.9379042690815006, 0.8829663962920047, 0.9577464788732394, 0.8981132075471698, 0.9436795994993742, 0.939729397293973, 0.9214536928487691, 0.9327731092436975, 0.9586374695863747, 0.9630952380952381, 0.941747572815534, 0.9525504151838672, 0.9698067632850241, 0.928395061728395, 0.9530026109660574, 0.9305724725943971, 0.9302325581395349, 0.9267973856209151, 0.9221411192214112]
F1-Scores
[0.9425406186796635, 0.92194606631211, 0.9285754714114418, 0.9144322434790408, 0.9572985216162164, 0.9125161404276809, 0.9368758667869853, 0.869179425603918, 0.9566889219191961, 0.8929239725659164, 0.9454824956254007, 0.9421590254289267, 0.9232611773066305, 0.934477043678535, 0.9584586070827081, 0.961298850954128, 0.9422072969584032, 0.952235274063853, 0.9711760527952288, 0.9300092280821304, 0.95161588

##### 2b: Unprocessed

In [11]:
data_path = Path("../data/sat1/split_stage_data_unprocessed.nc")
data = xr.load_dataset(data_path)

In [12]:
model = SAT1GRU
model_kwargs = {
    "n_channels": len(data.channels),
    "n_samples": len(data.samples),
    "n_classes": len(data.labels),
}
train_kwargs = {
    "logs_path": logs_path,
    "additional_info": {"preprocessing": "unprocessed"},
    "additional_name": f"preprocessing-unprocessed",
    "labels": SAT1_STAGES_ACCURACY,
}
results = k_fold_cross_validate(
    model, model_kwargs, data, n_folds, train_kwargs=train_kwargs, normalization_fn=normalization_fn
)
print_results(results)

Fold 1: test fold: ['0009']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

...

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

  0%|          | 0/153 [00:00<?, ? batch/s]

Fold 25: Accuracy: 0.9343065693430657
Fold 25: F1-Score: 0.9341090517191237
Accuracies
[0.9354838709677419, 0.9290012033694344, 0.9561611374407583, 0.9090909090909091, 0.9273399014778325, 0.912883435582822, 0.963777490297542, 0.9107763615295481, 0.9636150234741784, 0.9056603773584906, 0.9649561952440551, 0.9606396063960639, 0.9003516998827668, 0.9117647058823529, 0.9720194647201946, 0.9630952380952381, 0.9466019417475728, 0.9454329774614472, 0.9782608695652174, 0.9543209876543209, 0.9712793733681462, 0.925700365408039, 0.9011627906976745, 0.930718954248366, 0.9343065693430657]
F1-Scores
[0.9378124790852738, 0.9239112499743962, 0.9550049415536099, 0.9091232165016475, 0.930245241395086, 0.9176586157622648, 0.9650092895343624, 0.9022705721137353, 0.9587673626707531, 0.9052853743476547, 0.9644559882938261, 0.9627964113629446, 0.90309624552674, 0.9113146767590609, 0.9707325725326811, 0.9620252321133842, 0.9477816312533973, 0.9463028008376781, 0.9793240178405274, 0.9559969797445007, 0.970624

In [ ]:
# View results in Tensorboard
! tensorboard --logdir ../logs/exp_preprocessing/